---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

# Information

*get_list_of_university_towns():*
    
    Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. 

*get_recession_start / end / bottom():*

    Returns the year and quarter of the recession start / end / bottom time as a 
    string value in a format such as 2005q3
    
*convert_housing_data_to_quarters():*

    Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
*run_ttest():*

    First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).

In [3]:
def get_list_of_university_towns():
    import pandas as pd
    import numpy as np
    
    Townsdf = (pd.read_table('university_towns.txt',
                           header= None,
                           encoding='utf_8'
                          )
             [0]
             .tolist()
             )

    Towns = []
    States = []
    StatesCheck = []
    
    for Object in Townsdf:
    
        if '[ed' in Object:
            State = Object
            Index = State.index( '[' )
            State = State[ :Index ]
            States += [ State ]
            StatesCheck += [ State ]
        
        else:
            Town = Object
        
            if ' (' in Town:
                Index = Town.index( ' (' )
                Town = Town[ :Index ]
            
            State = States[-1]
            States += [ State ]
        
            Towns += [ Town ]
        
    for State in StatesCheck:
        State = States.index( State )
        del States[ State ]
    

    
    
    DataFrame = pd.DataFrame( {'RegionName': Towns,
                               'State': States,
                               }  
                            )
    
    DataFrame = DataFrame[ ['State','RegionName'] ]
    
    return DataFrame

In [4]:
def get_recession_start():
    import pandas as pd
    import numpy as np
    
    GDP = pd.read_excel( 'gdplev.xls',
                        header= None,
                        encoding='utf_8'
                       )

    WantedColumns = ['Year', 'GDP', 'GDP (2009)']


    GDPY = (GDP
            .drop([ Number for Number in range(3,8)], axis= 1)
            .rename( columns= { GDP.columns[ Index ]: Column for Index, Column in enumerate(WantedColumns) } )
            [8:]
            .reset_index()
            .drop('index', axis= 1)
            .dropna()
            )

    GDPQ = (GDP
            .drop([ Number for Number in range(4)], axis= 1)
            .drop(7, axis= 1)
            .rename( columns= { GDP.columns[ Index + 4 ]: Column for Index, Column in enumerate(WantedColumns) } )
            [8:]
            .reset_index()
            .drop('index', axis= 1)
            .dropna()
            )
    
    Filter = GDPQ['Year']
    
    for Index, Year in enumerate( Filter ): 
        if Year[:2] != '20':
            Filter[ Index ] = None
            
    GDPQ = (GDPQ
            .dropna()
            .reset_index()
            .drop( 'index', axis= 1)
           )

    Recessions = []
    
    for QuarterIndex in GDPQ.index.tolist():  
        Quarter = GDPQ.loc[ QuarterIndex ]
        if QuarterIndex != 0:
            if Quarter['GDP'] < GDPQ.loc[ QuarterIndex - 1 ][ 'GDP' ]:
                Recessions += [ QuarterIndex ]
    
    RecessionStart = []
    
    for RIndex, Index in enumerate( Recessions ):
        if Index != Recessions[ -1 ]:
            if ( (Index + 1) == Recessions[ RIndex + 1 ] ):
                RecessionStart += [ Index ]
    
    for RIndex, Index in enumerate( RecessionStart ):
        if Index != RecessionStart[ -1 ]:
            if ( (Index + 1) != RecessionStart[ RIndex + 1 ] ):
                 del RecessionStart[ RIndex ]

    RecessionStart = RecessionStart[0] - 1

    RecessionEnd = []

    Recessions = []

    for QuarterIndex in GDPQ.index.tolist()[ RecessionStart + 1 :]:  
        Quarter = GDPQ.loc[ QuarterIndex ]
        if QuarterIndex != 0:
            if Quarter['GDP'] > GDPQ.loc[ QuarterIndex - 1 ][ 'GDP' ]:
                Recessions += [ QuarterIndex ]
            
    for RIndex, Index in enumerate( Recessions ):
        if Index != Recessions[ -1 ]:
            if ( (Index + 1) == Recessions[ RIndex + 1 ] ):
                RecessionEnd += [ Index ]
    
    for RIndex, Index in enumerate( RecessionEnd ):
        if Index != RecessionEnd[ -1 ]:
            if ( (Index + 1) != RecessionEnd[ RIndex + 1 ] ):
                del RecessionEnd[ RIndex ]
                
    RecessionEnd = RecessionEnd[0] - 1

    return GDPQ[ 'Year' ][ RecessionStart ]

In [5]:
def get_recession_end():
    import pandas as pd
    import numpy as np
    
    GDP = pd.read_excel( 'gdplev.xls',
                        header= None,
                        encoding='utf_8'
                       )

    WantedColumns = ['Year', 'GDP', 'GDP (2009)']


    GDPY = (GDP
            .drop([ Number for Number in range(3,8)], axis= 1)
            .rename( columns= { GDP.columns[ Index ]: Column for Index, Column in enumerate(WantedColumns) } )
            [8:]
            .reset_index()
            .drop('index', axis= 1)
            .dropna()
            )

    GDPQ = (GDP
            .drop([ Number for Number in range(4)], axis= 1)
            .drop(7, axis= 1)
            .rename( columns= { GDP.columns[ Index + 4 ]: Column for Index, Column in enumerate(WantedColumns) } )
            [8:]
            .reset_index()
            .drop('index', axis= 1)
            .dropna()
            )
    
    Filter = GDPQ['Year']
    
    for Index, Year in enumerate( Filter ): 
        if Year[:2] != '20':
            Filter[ Index ] = None
            
    GDPQ = (GDPQ
            .dropna()
            .reset_index()
            .drop( 'index', axis= 1)
           )

    Recessions = []
    
    for QuarterIndex in GDPQ.index.tolist():  
        Quarter = GDPQ.loc[ QuarterIndex ]
        if QuarterIndex != 0:
            if Quarter['GDP'] < GDPQ.loc[ QuarterIndex - 1 ][ 'GDP' ]:
                Recessions += [ QuarterIndex ]
    
    RecessionStart = []
    
    for RIndex, Index in enumerate( Recessions ):
        if Index != Recessions[ -1 ]:
            if ( (Index + 1) == Recessions[ RIndex + 1 ] ):
                RecessionStart += [ Index ]
    
    for RIndex, Index in enumerate( RecessionStart ):
        if Index != RecessionStart[ -1 ]:
            if ( (Index + 1) != RecessionStart[ RIndex + 1 ] ):
                 del RecessionStart[ RIndex ]

    RecessionStart = RecessionStart[0] - 1

    RecessionEnd = []

    Recessions = []

    for QuarterIndex in GDPQ.index.tolist()[ RecessionStart + 1 :]:  
        Quarter = GDPQ.loc[ QuarterIndex ]
        if QuarterIndex != 0:
            if Quarter['GDP'] > GDPQ.loc[ QuarterIndex - 1 ][ 'GDP' ]:
                Recessions += [ QuarterIndex ]
            
    for RIndex, Index in enumerate( Recessions ):
        if Index != Recessions[ -1 ]:
            if ( (Index + 1) == Recessions[ RIndex + 1 ] ):
                RecessionEnd += [ Index ]
    
    for RIndex, Index in enumerate( RecessionEnd ):
        if Index != RecessionEnd[ -1 ]:
            if ( (Index + 1) != RecessionEnd[ RIndex + 1 ] ):
                del RecessionEnd[ RIndex ]
                
    RecessionEnd = RecessionEnd[0] + 1

    return GDPQ['Year'][RecessionEnd]

In [6]:
def get_recession_bottom():
    import pandas as pd
    import numpy as np
    
    GDP = pd.read_excel( 'gdplev.xls', 
                        header= None,
                        encoding='utf_8'
                       )

    WantedColumns = ['Year', 'GDP', 'GDP (2009)']


    GDPY = (GDP
            .drop([ Number for Number in range(3,8)], axis= 1)
            .rename( columns= { GDP.columns[ Index ]: Column for Index, Column in enumerate(WantedColumns) } )
            [8:]
            .reset_index()
            .drop('index', axis= 1)
            .dropna()
            )

    GDPQ = (GDP
            .drop([ Number for Number in range(4)], axis= 1)
            .drop(7, axis= 1)
            .rename( columns= { GDP.columns[ Index + 4 ]: Column for Index, Column in enumerate(WantedColumns) } )
            [8:]
            .reset_index()
            .drop('index', axis= 1)
            .dropna()
            )
    
    Filter = GDPQ['Year']
    
    for Index, Year in enumerate( Filter ): 
        if Year[:2] != '20':
            Filter[ Index ] = None
            
    GDPQ = (GDPQ
            .dropna()
            .reset_index()
            .drop( 'index', axis= 1)
           )

    RecessionStart = get_recession_start()
    RecessionEnd = get_recession_end()
    
    GDPTest = (GDPQ
               .set_index( 'Year' )
               .loc[ RecessionStart: RecessionEnd ]
               .reset_index()
              )
    
    GDPBottom =(GDPTest
                .where( GDPTest['GDP'] == GDPTest['GDP'].min() )
                .dropna()
                ['Year']
                .unique()[0]
               )
    
    
    return GDPBottom

In [7]:
def convert_housing_data_to_quarters():
    import pandas as pd
    import numpy as np
    
    Zillow = pd.read_csv( 'City_Zhvi_AllHomes.csv' )

    Zillow['State'] = Zillow['State'].replace( states )

    WantedColumns = Zillow.columns.tolist()[:5] + list( filter( lambda x: x if str( x )[:2] == '20' else False,
                                                               Zillow.columns.tolist()[5:]
                                                              ) 
                                                      )

    Zillow = Zillow[WantedColumns]

    Years = Zillow.columns.tolist()[5:]

    Quarters = {}

    for Year in Years:
        Quarter = Year[:4] + 'q'
        Period = int(Year[5:])
    
        if  Period in range( 1, 3 +1 ):
            Quarter += '1'
        elif Period in range( 4, 6 +1 ):
            Quarter += '2'
        elif Period in range( 7, 9 +1 ):
            Quarter += '3'
        else:
            Quarter += '4'
    
        if Quarter not in Quarters.keys():
            Quarters[ Quarter ] = [ Year ]
        else:
            Quarters[ Quarter ] += [ Year ]
    

    for Quarter in Quarters:
        Zillow[ Quarter ] = Zillow.T.loc[ Quarters[ Quarter ] ].mean()
    
    Zillow = Zillow.set_index( ['State', 'RegionName'] )
    
    WantedColumns = list( Quarters.keys() )

    Zillow = Zillow[ WantedColumns ]

    return Zillow

In [8]:
def run_ttest():
    return (True, 0.0054964273536330264, 'university town')